In [ ]:
from nbodykit.lab import *
import pandas as pd
import numpy as np

zmin, zmax = 0.01, 0.15

# Load your data and randoms as Pandas DataFrames
sdss = pd.read_csv('../data/sdss_zmin_0.000_zmax_0.300_sigma_5.0.csv')
sdss  = sdss[(sdss["red"] > zmin)&(sdss["red"] < zmax)]
randoms = pd.read_csv('../data/random_sample_healpy_128.csv')  # Replace with your randoms CSV file path

# Convert DataFrames to ArrayCatalog with RA, DEC, and Redshift
data_cat = ArrayCatalog({'RA': sdss['ra'].values,
                         'DEC': sdss['dec'].values,
                         'Redshift': sdss['red'].values})

rand_cat = ArrayCatalog({'RA': randoms['ra'].values,
                         'DEC': randoms['dec'].values,
                         'Redshift': randoms['red'].values})

data_cat['RA'] = np.radians(data_cat['RA'])
data_cat['DEC'] = np.radians(data_cat['DEC'])
rand_cat['RA'] = np.radians(rand_cat['RA'])
rand_cat['DEC'] = np.radians(rand_cat['DEC'])

# Assign cosmology (if not already in the data)
cosmo = cosmology.Planck15

# Define the edges for s and mu bins
s_bins = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]  # smaller range for simplicity
mu_bins = 10  # fewer bins for testing

# Define the SurveyData2PCF object
result = SurveyData2PCF('2d', data_cat, rand_cat, edges=s_bins, Nmu=mu_bins, cosmo=cosmo, ra='RA', dec='DEC', redshift='Redshift')

# Access the computed correlation function
xi_s_mu = result.corr.data  # 2D correlation function values
s = result.corr.coords['r']  # The s bins
mu = result.corr.coords['mu']  # The mu bins

# Save or print results
print("s:", s)
print("mu:", mu)
print("xi(s, mu):", xi_s_mu)

# Optionally save the results to a file
np.savez('xi_s_mu_results.npz', s=s, mu=mu, xi_s_mu=xi_s_mu)



In [44]:
test_data = ArrayCatalog({'RA': np.array([10, 20, 30]), 'DEC': np.array([5, 15, 25]), 'Redshift': np.array([0.1, 0.2, 0.3])})
test_randoms = ArrayCatalog({'RA': np.array([10, 20, 30]), 'DEC': np.array([5, 15, 25]), 'Redshift': np.array([0.1, 0.2, 0.3])})

# Try SurveyData2PCF with this small test data
result_test = SurveyData2PCF('2d', test_data, test_randoms, edges=s_bins, Nmu=mu_bins, cosmo=cosmo)
print(result_test)



RuntimeError: calling the function 'Corrfunc.mocks.DDsmu_mocks.DDsmu_mocks' failed, likely due to issues with input data/parameters. Open at issue at https://github.com/bccp/nbodykit/issues for further help.
exception: RuntimeError occurred
stdout: 
stderr: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Load the results (if saved) or use the variables directly
# data = np.load('xi_s_mu_results.npz')
# s = data['s']
# mu = data['mu']
# xi_s_mu = data['xi_s_mu']

# Ensure s and mu are arrays, not edges
s_centers = 0.5 * (s[1:] + s[:-1])  # Bin centers for s
mu_centers = mu  # Mu is already centered if Nmu was provided

# Create a 2D grid for plotting
s_grid, mu_grid = np.meshgrid(s_centers, mu_centers, indexing='ij')

# Plotting
plt.figure(figsize=(8, 6))
plt.pcolormesh(mu_grid, s_grid, xi_s_mu.T, shading='auto', cmap='viridis')
plt.colorbar(label=r'$\xi(s, \mu)$')
plt.xlabel(r'$\mu$ (cosine of line of sight angle)')
plt.ylabel(r'$s$ (Mpc/h)')
plt.title('2D Correlation Function: $\xi(s, \mu)$')
plt.tight_layout()
plt.show()
